# Accessing Allen Institute Database using API

### Goals
1. Understand what is Application Programming Interface (API).
2. Learn how the API package provided by Allen Institude retrieves data.
3. Write a program to retrieve a cell from Allen Institute database.

### Introduction
The [allensdk.api](https://allensdk.readthedocs.io/en/latest/allensdk.api.html#module-allensdk.api) package is designed to help retrieve data from the [Allen Brain Atlas API](http://help.brain-map.org/display/api/Allen+Brain+Atlas+API) contains methods to help formulate API queries and parse the returned results. There are several pre-made subclasses available that provide pre-made queries specific to certain data sets. We will use following subclasses in Allen SDK for this project:  
&emsp;__CellTypesApi:__ data related to the Allen Cell Types Database  
&emsp;__RmaApi:__ general-purpose HTTP interface to the Allen Institute API data model and services

__Some useful links to the Allen website:__  
&emsp;__[Install guide](https://allensdk.readthedocs.io/en/latest/install.html)__  
&emsp;__[Introduction to the API Access](https://allensdk.readthedocs.io/en/latest/data_api_client.html)__  
&emsp;__[Allen Brain Atlas API - Allen Cell Types Database](http://help.brain-map.org/display/celltypes/API)__  
&emsp;__[Example jupyter notebook - Introduction to the Cell Types Database](https://allensdk.readthedocs.io/en/latest/_static/examples/nb/cell_types.html)__  
&emsp;__[Source documentation of allensdk.api.queries.cell_types_api module](https://allensdk.readthedocs.io/en/latest/allensdk.api.queries.cell_types_api.html)__  

### Procedure
#### 1. Install the API package.  
If you have Anaconda installed, just enter ```pip install allensdk``` in an Anaconda Prompt or in a terminal.To uninstall, enter ```pip uninstall allensdk```.

#### 2. Download all cells
Import ```CellTypeApi``` class. If you want to download Cell Types Database data to a standard directory structure on your hard drive, use ```CellTypeCache``` class instead. See the example jupyter notebook given in the links.  
It may take a few seconds to download the data and return a list of all cells.

In [ ]:
from allensdk.api.queries.cell_types_api import CellTypesApi

cta = CellTypesApi() # the CellTypesApi instance

cells = cta.list_cells_api() # Query the API for a list of all cells in the Cell Types Database.

```specimen__id``` is a unique id for each cell recorded in the database. 

In [ ]:
specimen_id_list = [cell['specimen__id'] for cell in cells] # store the specimen IDs in a list
print(specimen_id_list[:10]) # displace the first 10 IDs

idx = 0  # select an index for an ID from the list
cell_id = specimen_id_list[idx] # the ID of the first cell from the list
print(cell_id)

#### 2. Get a single cell
Several ways to get a cell according to its ID.

In [ ]:
cell = cells[idx]            # Get the selected cell from the list of all cells
cell = cta.get_cell(cell_id) # A method to retrieve a single cell from the database, returning the same cell as in the line above
cell

Convert cell list into a pandas DataFrame and get the cell from it.

In [ ]:
import pandas as pd
cells_df = pd.DataFrame(cells)
cell_df = cells_df[cells_df.specimen__id==cell_id] # get a cell according to its ID
cell_df

#### 3. Get electrophysiology features
Download electrophysiology features for all cells. This may take a few seconds to download.

In [ ]:
ephys_features = cta.get_ephys_features()

Convert it into pandas DataFrame and get the features for a cell according to its ID.

In [ ]:
import pandas as pd
ef_df = pd.DataFrame(ephys_features)

cell_ef = ef_df[ef_df.specimen_id==cell_id]
cell_ef

Another way to download the electrophysiology features for a single cell is by using ```rma_api``` which are the bases of ```cell_types_api```.

In [ ]:
from allensdk.api.queries.rma_api import RmaApi

rma = RmaApi() # the RmaApi instance

data = rma.model_query(model='EphysFeature',criteria='[specimen_id$eq'+str(cell_id)+']')[0] # or use criteria='[specimen_id$eq525011903]'
data

Following line use RmaApi to find the IDs of cells whose electrophysioloy recording somehow failed.

In [ ]:
noEFcells = rma.model_query(model='EphysFeature',criteria='specimen(ephys_result[failed$eqtrue])',num_rows='all')

noEF_id = [cell['specimen_id'] for cell in noEFcells]
print(noEF_id) # display the IDs
noEFcell = rma.model_query(model='EphysFeature',criteria='[specimen_id$eq'+str(noEF_id[0])+']')[0]
noEFcell # display an example of the electrophysiology features from a failed recording.

#### 4. Get cells according to species
Check the key ```donor__species``` for all cells in the data. We can see that there are two species that the cells are collected from, human and mouse.

In [ ]:
print(set([cell['donor__species'] for cell in cells]))

We can use the following method ```filter_cells_api``` to get a desired subset from the database. Keyword argument ```require_morphology``` should be ```Ture``` if you want to filter out cells that have no morphological images.Similarly keyword argument ```require_reconstruction``` is for filtering out cells with no morphological reconstruction. Keyword argument ```species``` specify the filter for cells that belong to one or more species.

In [ ]:
mousecells = cta.filter_cells_api(cells,require_morphology=True,require_reconstruction=True,species=[CellTypesApi.MOUSE]) # mouse cells with morphological images
print(len(mousecells))
humancells = cta.filter_cells_api(cells,require_morphology=False,require_reconstruction=False,species=[CellTypesApi.HUMAN]) # all human cells
print(len(humancells))

mousecells[0] # display the first cell in the filtered list of mouse cells for example

In the returned cell, the key ```id``` denotes the specimen ID mentioned above. Using ```id``` value will help you find a particular cell from the filtered list.

In [ ]:
ID = mousecells[0]['id'] # the specimen id of the example mouse cell
ID

#### 5. For a single cell save its electrophysology recordings as an NWB file and its morphological reconstruction as a SWC file in current directory

In [ ]:
cta.save_ephys_data(specimen_id=cell_id,file_name='ephys_'+str(cell_id)+'.nwb')

In [ ]:
cta.save_reconstruction(specimen_id=ID,file_name='reconstruction_'+str(ID)+'.swc')

### 6. Task: Retrieve a cell
Retrieve a __human__ cell with both __electrophysiology__ recordings and __morphological__ reconstruction. Then save them as NWB file and SWC file. Display the cell attributes in jupyter notebook. And try to save its electrophysiology features from a pandas DataFrame to a csv file using pandas method ```your_dataframe.to_csv(path_or_buf='your_file_name.csv')```.